### Imports

In [2]:
from caveclient import CAVEclient
from intern import array
import pickle
import numpy as np
from agents import data_loader
from cloudvolume import CloudVolume
from membrane_detection import membranes
from agents.scripts import precompute_membrane_vectors, create_post_matrix, merge_paths, get_soma
import agents.sensor
from agents.run import run_agents
import aws.sqs as sqs
import sys
import time
import ast
import pandas as pd
import agents.scripts as scripts
from drive import drive
from finding_orphans import *
from math import sqrt
from tip_finding.tip_finding import tip_finder_decimation

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Helper functions


In [3]:
def find_euclidian_distance(proposed_endpoint, gt_endponts_array):
    diffs = gt_endponts_array - proposed_endpoint
    diffs_distance = np.sqrt(np.sum(np.square(diffs), axis=1))
    min_dist_ind = np.argmin(diffs_distance)
    min_dist = diffs_distance[min_dist_ind]
    return min_dist

In [32]:
def endpoint_generator(load, invalidation_d, humfrey_iters, decimation_factor):
    for idx, i in enumerate(load['seg_id']):
        # if idx == 20:
        #     break
        try:
            t1, skel, mesh_obj = tip_finder_decimation(str(i))
            endpoints_dict[i] = t1
        except:
            print(f"\n\nSeg {i} returned error on get. Skipping.\n")
            pass
    load['endpoints'] = load['endpoints'].apply(
        lambda x: list(ast.literal_eval(x)))
    load['proposed_endpoints'] = load.seg_id.map(endpoints_dict)
    return load


In [33]:
def testing_metrics(load, threshold, run, output):
    for index, row in load.iterrows():
        # if index == 20:
        #     break

        endpoint_array = np.array(row["endpoints"])
        proposed_endpoints_array = np.array(row["proposed_endpoints"])
        segID = row["seg_id"]

        #skip anything seg_id that isn't 'good'
        if row["comments"] != 'good':
            print("skipping \n")
            continue

        #if both say no endpoints, then it's correct
        elif (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) == 0):
            output.loc[len(output.index)] = [run, 1.0,1.0,1.0, segID, len(
                endpoint_array), endpoint_array, row["comments"], proposed_endpoints_array]
            continue

        #if we propose no endpoints but there are endpoints, it's wrong
        elif (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) > 0):
            output.loc[len(output.index)] = [run, 0.0,0.0,0.0, segID, len(
                endpoint_array), endpoint_array, row["comments"], proposed_endpoints_array]
            continue

        #if we propose endpoints but there are none, it's wrong
        elif proposed_endpoints_array.size > 0 and endpoint_array.size == 0:
            output.loc[len(output.index)] = [run, 0.0,0.0,0.0, segID, len(
                endpoint_array), endpoint_array, row["comments"], proposed_endpoints_array]
            continue

        #should skip for now if the endpoint array is size zero to avoid divide by zero error
        elif endpoint_array.size == 0:
            output.loc[len(output.index)] = [run,-1,-1,-1,segID, len(endpoint_array), endpoint_array, row["comments"], proposed_endpoints_array]
            continue

        else:
            endpoint_ids = np.arange(0, len(endpoint_array))
            test_ids = np.arange(
                0, len(proposed_endpoints_array)) + len(proposed_endpoints_array)

            #get precision and recall at end of each iteration
            analysis = run_synapse_analysis(
                endpoint_array,
                np.array(endpoint_ids),
                proposed_endpoints_array,
                np.array(test_ids),
                threshold,
                iso_correction=10,
            )

            output.loc[len(output.index)] = [run, analysis.precision, analysis.recall, analysis.f1, segID, len(
                endpoint_array), endpoint_array, row["comments"], proposed_endpoints_array]
    return output


In [6]:
"""
Various utility classes and functions for Confirms.
"""
from copy import deepcopy

import numpy as np
import pandas as pd
from scipy import spatial
from scipy.optimize import linear_sum_assignment


def calculate_precision_recall(tp, fp, fn):
    """
    Calculate precision/recall from given true positives, false positives, and false negatives.
    Parameters
    ----------
    tp : int
        The number of true positives.
    fp : int
        The number of false positives.
    fn : int
        The number of false negatives.
    Returns
    -------
    precision : float
        The precision score.
    recall : float
        The recall score.
    """
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return precision, recall


def calculate_f1(precision, recall):
    """
    Calculate the F1 score from precision/recall scores.
    Parameters
    ----------
    precision : float
        The precision score.
    recall : float
        The recall score.
    Returns
    -------
    f1 : float
        The F1 score.
    """
    return 2 * ((precision * recall) / (precision + recall))


def get_summary_metrics(array):
    """
    Calculate a number of summary metrics on an array of numbers.
    Parameters
    ----------
    array : array_like
        Array containing numbers who summary metrics is desired.
    Returns
    -------
    metrics : dict
        Dict containing the mean, median, max, min, range, standard deviation,
        and variance of the input array.
    """
    summary_object = {}
    array = np.array(array)

    summary_object["mean"] = np.mean(array)
    summary_object["median"] = np.median(array)
    summary_object["max"] = np.amax(array)
    summary_object["min"] = np.amin(array)
    summary_object["range"] = np.mean(array)
    summary_object["stddev"] = np.std(array)
    summary_object["variance"] = np.var(array)

    return summary_object


def munkres_assignment(workers, jobs):
    """
    Perform hungarian-munkres assignment.
    Parameters
    ----------
    workers : array_like
        Array containing the first set of points.
    jobs : array_like
        Array containing the second set of points.
    Returns
    -------
    cost_matrix : numpy.ndarray
        Matrix containing pairwise distances (the cost of assignment).
    row_ind : numpy.ndarray
        Row indices of cost_matrix for optimal assignment.
    col_ind : numpy.ndarray
        Column indices of cost_matrx for optimal assignment.
    """
    cost_matrix = spatial.distance.cdist(workers, jobs, "euclidean")
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    return cost_matrix, row_ind, col_ind


def make_isotropic(xyz, correction, dimen=2):
    """
    Correct anisotropy in a collection of x, y, z coordinates.
    This function performs a deepcopy of xyz before making the necessary modifications.
    Parameters
    ----------
    xyz : numpy.ndarray or pandas.DataFrame:
        The coordinate values.
    correction : float
        The value to correct anisotrophy.
    dimen : int
        Index of last dimension to which to apply the correction. Default is 2.
    Returns
    -------
    iso_xyz : numpy.ndarray or pandas.DataFrame
        An isotropic version of xyz.
    """

    if not isinstance(xyz, np.ndarray) and not isinstance(xyz, pd.DataFrame):
        xyz = np.asarray(xyz)

    shape = np.shape(xyz)
    isotropic_xyz = xyz.copy()
    size = shape[-1]
    if dimen >= size or dimen < 0:
        raise ValueError(
            "improper dimen value (valid: 0 through {})".format(size - 1))
    if isinstance(xyz, np.ndarray):
        isotropic_xyz[..., dimen] = isotropic_xyz[..., dimen] * correction
    else:
        isotropic_xyz.iloc[:, dimen] = isotropic_xyz.iloc[:,
                                                          dimen] * correction
    return isotropic_xyz


In [16]:
"""
Confirms synapse processing and analysis functions.
"""
from collections import namedtuple

import numpy as np
import pandas as pd
from scipy import spatial
from scipy.optimize import linear_sum_assignment

# from . import utils
# import utils

SynapseMetrics = namedtuple(
    "SynapseMetrics",
    ["precision", "recall", "f1", "tp_gt_ids", "tp_test_ids", "fp_ids", "fn_ids"],
)


def filter_synapse_id_core(volume, xyz, ids, box_radius_nm=2500):
    """
    Filter synapses to only return those within a central core.
    Parameters
    ----------
    volume : dict
        The volume to filter on.
    xyz : array_like
        Synapse xyz coordinates.
    ids : array_like
        Synapse hash (ids).
    box_radius_nm : int
        radius of cube, from volume core.
    Returns
    -------
        xyz : numpy.ndarray
            Synapse coordinates.
        ids : numpy.ndarray
            Synapse ids.
    """

    xyz_out = []
    id_out = []
    center = np.asarray(volume["center"], "float")
    base_resolution = np.asarray(volume["base_resolution"], "float")
    annotation_resolution = np.asarray(volume["resolution"], "float")

    pad_vx = box_radius_nm / base_resolution[0] / (2 ** annotation_resolution)
    pad_vy = box_radius_nm / base_resolution[1] / (2 ** annotation_resolution)
    pad_vz = box_radius_nm / base_resolution[2]
    xr = (center[0] - pad_vx, center[0] + pad_vx)
    yr = (center[1] - pad_vy, center[1] + pad_vy)
    zr = (center[2] - pad_vz, center[2] + pad_vz)

    for i in range(len(xyz)):
        x = xyz[i][0]
        y = xyz[i][1]
        z = xyz[i][2]

        if (
            x > xr[0]
            and x < xr[1]
            and y > yr[0]
            and y < yr[1]
            and z > zr[0]
            and z < zr[1]
        ):
            xyz_out.append(xyz[i])
            id_out.append(ids[i])

    return np.array(xyz_out), np.array(id_out, dtype=np.object)


def synapse_match(
    xyz_truth, xyz_detect, id_truth, id_detect, thresh
):  # pylint: disable=R0914
    """
    <Description here>
    Parameters
    ----------
    xyz_truth : array_like
        <description>
    xyz_detect : array_like
        <description>
    id_truth : array_like
        <description>
    id_detech : array_like
        <description>
    thresh : float
        <description>
    Returns
    -------
    id_lookup : <type>
        <description>
    """

    # pylint: disable=C0103

    # Ensure we have numpy arrays
    xyz_truth = np.asarray(xyz_truth)
    xyz_detect = np.asarray(xyz_detect)
    id_truth = np.asarray(id_truth)
    id_detect = np.asarray(id_detect)

    cost, row_ind, col_ind = munkres_assignment(xyz_truth, xyz_detect)
    print(cost)
    match_idx = np.where(cost[row_ind, col_ind] < thresh)

    if len(match_idx) > 0:
        # row is idx of GT TP
        # col is idx of student TP
        gt_tp_idx, det_tp_idx = row_ind[match_idx], col_ind[match_idx]
        gt_tp_ids, det_tp_ids = id_truth[gt_tp_idx], id_detect[det_tp_idx]
        # Combine into pairs
        id_lookup = np.column_stack((gt_tp_ids, det_tp_ids))
    else:
        gt_tp_idx = []
        det_tp_idx = []
        id_lookup = np.column_stack(
            (np.array([], dtype="object"), np.array([], dtype="object"))
        )

    # not in row (set diff) are FN
    gt_syn_idx = np.arange(0, len(xyz_truth))
    fn_idx = np.setdiff1d(gt_syn_idx, gt_tp_idx)
    if len(fn_idx) > 0:
        fn_ids = id_truth[fn_idx]
        id_lookup_fn = np.column_stack((fn_ids, np.repeat(None, len(fn_ids))))
        id_lookup = np.concatenate((id_lookup, id_lookup_fn))

    # not in col (set diff) are FP
    det_syn_idx = set(np.arange(0, len(xyz_detect)))
    fp_idx = np.asarray(list(det_syn_idx.difference(det_tp_idx)))
    if len(fp_idx) > 0:
        fp_ids = id_detect[fp_idx]
        id_lookup_fp = np.column_stack((np.repeat(None, len(fp_ids)), fp_ids))
        id_lookup = np.concatenate((id_lookup, id_lookup_fp))

    return pd.DataFrame(id_lookup, columns=["ground_truth", "detect"])


def run_synapse_analysis(
    gt_xyzs,
    gt_ids,
    test_xyzs,
    test_ids,
    threshold,
    iso_corrected=False,
    iso_correction=1,
):
    """
    <Description here>
    Parameters
    ----------
    gt_xyzs : numpy.ndarray
        Array of ground truth xyz coordinates
    gt_ids : numpy.ndarray
        Array of ids associated with ground truth xyz coordinates
    test_xyzs : numpy.ndarray
        Array of test xyz coordinates
    test_ids : numpy.ndarray
        Array of ids associated with test xyz coordinates
    threshold : float
        Synapse matching threshold
    iso_corrected : boolean
        Mark whether the data is isotropic. If not, it will be made isotropic using the
        `iso_correction` parameter.
    iso_correction : float
        Value to correct anistropy.
    Returns
    -------
    sm : SynapseMetrics
        Resultant object containing precision, recall, and F1 scores, along with
        with true positive (both ground truth and test), false positive, and false negative
        ids.
    """
    # pylint: disable=R0913,R0914
    if not iso_corrected:
        # gt_xyzs = utils.make_isotropic(gt_xyzs, iso_correction)
        gt_xyzs = make_isotropic(gt_xyzs, iso_correction)
        # test_xyzs = utils.make_isotropic(test_xyzs, iso_correction)
        test_xyzs = make_isotropic(test_xyzs, iso_correction)

    results_table = synapse_match(
        gt_xyzs, test_xyzs, gt_ids, test_ids, threshold)

    tp = results_table.dropna()
    fn = results_table[results_table.detect.isnull()]
    fp = results_table[results_table.ground_truth.isnull()]

    assert len(tp.ground_truth) == len(
        tp.detect
    ), "true positive ground truth and test size mismatch"

    tp_count = len(tp)
    fp_count = len(fp)
    fn_count = len(fn)

    try:
        # precision, recall = utils.calculate_precision_recall(
        #     tp_count, fp_count, fn_count
        # )

        precision, recall = calculate_precision_recall(
            tp_count, fp_count, fn_count
        )

    except ZeroDivisionError:
        precision, recall = np.nan, np.nan

    try:
        # f1 = utils.calculate_f1(precision, recall)
        f1 = calculate_f1(precision, recall)
    except ZeroDivisionError:
        f1 = np.nan

    sm = SynapseMetrics(
        precision=precision,
        recall=recall,
        f1=f1,
        tp_gt_ids=np.asarray(tp.ground_truth),
        tp_test_ids=np.asarray(tp.detect),
        fp_ids=np.asarray(fp.detect),
        fn_ids=np.asarray(fn.ground_truth),
    )
    return sm


### Running metrics

In [34]:
# Loading in data
load = pd.read_csv("agents/Data/endpoints_gt5.csv")
endpoints_dict = {}

# #creating series objects for the desired collumns
# segIDs = load['seg_id']
# Endpoints = load['endpoints']


In [35]:
# Generate out_df

out_df = pd.DataFrame(columns=["run", "precision", "recall", "f1", "seg_id",
                      "num_endpoints", "endpoints", "comments", "proposed_endpoints"])


In [36]:
run = 0
# for invalidation_d in range(4000, 6001, 200):
for invalidation_d in range(4000, 4201, 200):
    # for num_humfrey_iters in range(0,251,25):
    for num_humfrey_iters in range(50, 76, 25):
        # for decimation_factor in np.arange(0.3, 0.71, 0.04):
        for decimation_factor in np.linspace(0.3, 0.35, num = 2):
            load = pd.read_csv("agents/Data/endpoints_gt5.csv") #NOT THE BEST WAY TO DO THIS!!!
            run += 1
            load_with_proposed_endpoints = endpoint_generator(
                load, invalidation_d=invalidation_d, humfrey_iters=num_humfrey_iters, decimation_factor=decimation_factor)
            out_df = testing_metrics(
                load_with_proposed_endpoints, threshold=500, run=run, output=out_df)


07/29/2022 11:11:43 AM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 3996/3996 [00:00<00:00, 367311.83it/s]
07/29/2022 11:11:45 AM WARNING: face_normals incorrect shape, ignoring! 


  0%|          | 0/10649 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/meshparty/skeletonize.py:613: RuntimeWarning: invalid value encountered in multiply
  target = np.nanargmax(root_ds * valid)
100%|██████████| 10649/10649 [00:00<00:00, 471509.41it/s]
07/29/2022 11:11:48 AM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 1140/1140 [00:00<00:00, 244904.04it/s]
07/29/2022 11:11:53 AM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 91377/91377 [00:00<00:00, 189416.89it/s]
07/29/2022 11:12:04 AM WARNING: face_normals incorrect shape, ignoring! 


...

100%|██████████| 9618/9618 [00:00<00:00, 679561.61it/s]
07/29/2022 12:23:10 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 38734/38734 [00:00<00:00, 161337.97it/s]
07/29/2022 12:23:15 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 811/811 [00:00<00:00, 278111.40it/s]
07/29/2022 12:23:16 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 9313/9313 [00:00<00:00, 441543.11it/s]


[[  99.46356117 4423.90008929]
 [4415.78452826   24.37211521]]
[[ 146.85366866 5470.51569781 9272.57704201]
 [5554.85157317  495.77414213 8350.73439884]
 [9231.09576378 8664.4402589   368.27978495]]
skipping 

skipping 

[[173.70377083 226.46191733]
 [146.63560277 252.88337233]]
[[358.60842154 495.25548962]
 [491.02851241 233.07724042]]
[[  65.7267069  4494.89933146]
 [4475.84450132  624.97999968]]
[[901.23748258 517.58284361]
 [826.68313156 572.19926599]]
skipping 

[[648.0686692   81.63332653]
 [107.42904635 605.6153895 ]]
[[ 4128.18483113  3189.93291466    91.23595782]
 [  170.76592166  3482.68416598  4007.43771006]
 [11292.02129824  8929.37315829 11956.89278199]
 [13982.45489891 13733.27073206 16546.69710244]
 [ 7099.01887587  3574.97762231  6076.14853341]]
[[8128.64742746  864.88033854]
 [ 315.13171849 8280.50898194]]
[[  802.63628126  3717.38079836 10550.35264813]
 [10174.82270116  7734.51601589   253.21334878]
 [ 3197.35218579   939.4423878   8381.84043036]]
[[  657.3864921  123

In [37]:
out_df.to_csv('results.csv')


In [38]:
out_df

,run,precision,recall,f1,seg_id,num_endpoints,endpoints,comments,proposed_endpoints
0,1,1.000000,1.0,1.0,864691135909994000,2,"[[402584, 228856, 23991], [402985, 229235, 235...",good,"[[402567, 228758, 23991], [402990, 229222, 235..."
1,1,1.000000,1.0,1.0,864691135247440303,3,"[[401612, 224623, 23991], [405257, 226318, 236...",good,"[[401467, 224644, 23990], [405021, 226754, 236..."
2,1,1.000000,1.0,1.0,864691135314714227,2,"[[397867, 232230, 23999], [397854, 232252, 239...",good,"[[397815, 232393, 23996], [397931, 232013, 240..."
3,1,1.000000,1.0,1.0,864691135552841025,2,"[[403332, 227545, 24398], [403566, 227614, 244...",good,"[[403094, 227479, 24424], [403709, 227788, 244..."
4,1,0.500000,0.5,0.5,864691135516937065,2,"[[77262, 113088, 20442], [79178, 109116, 20402]]",good,"[[77206, 113116, 20440], [79748, 109356, 20411]]"
...,...,...,...,...,...,...,...,...,...
795,8,0.666667,1.0,0.8,864691136810680430,2,"[[116389, 195392, 19326], [121899, 191082, 178...",good,"[[116340, 195429, 19326], [118328, 192695, 189..."
796,8,0.500000,0.5,0.5,864691135089638001,2,"[[122789, 154701, 20649], [122688, 151314, 205...",good,"[[122634, 154535, 20597], [122670, 151366, 205..."
797,8,0.333333,0.5,0.4,864691135319600870,2,"[[106608, 111803, 20796], [98290, 97555, 21045]]",good,"[[98239, 97526, 21044], [104882, 104419, 20465..."
798,8,1.000000,1.0,1.0,864691136558839249,2,"[[103138, 188530, 21200], [91780, 189141, 20879]]",good,"[[91669, 189101, 20878], [103261, 188532, 21198]]"
